In [1]:
%run Utils/Utils.ipynb
%run Utils/Utils_Widgets.ipynb

Servo Motors Ready!
Lidar has started streaming data!
Object recognition model Ready!


In [2]:
# %run Utils/Utils_DepthCam.ipynb
%run Utils/Utils_Camera.ipynb

jetcam library imported
jetcam ready!
torchvision imported
Loading model...
FRCNN model ready!


# Parameters

In [3]:
goal_x_coor = -1.95 #-1.95  # in m, measured relative to the initial robot coordinates
goal_y_coor = 5.5 # 5.5  # in m, measured relative to the initial robot coordinatontrol_thread = False

running_control_thread = False

getObservation()
print(observation)

[5.835, -1.95, 5.5, -19.522, 0, 0.894, 0.886, 0.905, 0.954, 1.046, 1.184, 1.427, 1.847, 2.803, 2.739, 2.733, 2.798, 2.939, 0.001, 3.682, 4.406, 2.333, 2.43, 3.524, 5.544, 1.448, 1.526, 0.001, 1.225, 1.035, 0.924, 0.862, 0.841, 0.853, 0.892, 0.968, 1.07, 1.228, 1.121, 1.001, 0.93]


In [4]:
model_path = "Turtlebot3_0331.pth"
agent = torch.load(model_path)
agent.eval()
print('Model Loaded!')

Model Loaded!


# Control

In [5]:
def ControlRobot():
    step = 0
    delay_counter = 0
    while True:
        
        delay_counter+=1
        
        getObservation()

        # Get an Observation
        try:
            getObservation()
        except:
            continue

        updateStateWidgets()
                   
        # Get Action
        actions = GetAction(agent)
                
        if delay_counter>10:
        
#             # If still far away
            if observation[0] > 0.2:
                print(f'Performing action = {actions}')
                act(actions)

            else:
                print("reached goal")
                continue

            

In [6]:
def showDepthcamFeed():
    global clockIsVisible, color
    global running_control_thread
    
    while True:
        # Get a frame from the depthCamera
        color,_ = GetFrames(frontalCamera, getColor=True)
        
        # Normalize + send to CUDA
        image = torch.FloatTensor(color).transpose(0, 2)
        image = image[None,:]/255.0
        image = image.to(DEVICE)

        # Obtain BBoxes + Labels from model
        detections = model(image)[0]
        try:
            boxes, labels = detections['boxes'].detach().cpu().numpy(), detections['labels'].detach().cpu().numpy()
        except:
            boxes = [[0]]
            labels = [0]

        # Move all bboxes + labels that don't belong to a clock
        
        boxes = [box for box,label in zip(boxes,labels) if label == 85]
        labels = [label for label in labels if label == 85] # 85 ==> clock
        
        # If Clock was found, draw the bounding box
        if len(labels)>0:
            clockIsVisible = 1
            for idx,box in enumerate(boxes):
                cv2.rectangle(color, (box[0], box[2]), (box[1], box[3]), (255, 0, 0), 1)
        
        # If Clock not visible, let the State vector know about it.
        else:
            clockIsVisible = 0

            
        # Run Control Thread
        if running_control_thread == False:
            
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [7]:
def showDepthcamFeed_TRT_CSI():
    global clockIsVisible, image
    global running_control_thread
    
    while True:

        # Obtain BBoxes + Labels from model
        detections = getDetections(image)
        
        if 85 in detections:
            clockIsVisible = 1   
            
        else:
            clockIsVisible = 0
        
            
        # Run Control Thread
        if running_control_thread == False:
            
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [8]:
def showDepthcamFeed_TRT():
    global clockIsVisible, color
    global running_control_thread
    
    while True:
        # Get a frame from the depthCamera
        color,_ = GetFrames(frontalCamera, getColor=True)
        
        
        # Obtain BBoxes + Labels from model
        detections = getDetections(color)
        
        if 85 in detections:
            clockIsVisible = 1   
            
        else:
            clockIsVisible = 0
        
            
        # Run Control Thread
        if running_control_thread == False:
            
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [9]:
def showCSIFeed():
    global clockIsVisible, image
    global running_control_thread
    
    while True:
        
        # Normalize + send to CUDA
        image_ = torch.FloatTensor(image).transpose(0, 2)
        image_ = image_[None,:]/255.0
        image_ = image_.to(DEVICE)
        
        # Obtain BBoxes + Labels from model
        detections = model(image_)[0]
        try:
            boxes, labels = detections['boxes'].detach().cpu().numpy(), detections['labels'].detach().cpu().numpy()
        except:
            boxes = [[0]]
            labels = [0]

        # Move all bboxes + labels that don't belong to a clock
        
        boxes = [box for box,label in zip(boxes,labels) if label == 85]
        labels = [label for label in labels if label == 85] # 85 ==> clock
        
        
        # If Clock was found, draw the bounding box
        if len(labels)>0:
            clockIsVisible = 1
        
        # If Clock not visible, let the State vector know about it.
        else:
            clockIsVisible = 0
        
            
        # Run Control Thread
        if running_control_thread == False:
            controlThread = Thread(target = ControlRobot)
            controlThread.start()
            running_control_thread = True

In [10]:
display(full_widget)

In [11]:
# Delay Camera thread to make sure it works.
display(image_widget)

In [12]:
# Start camera
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
showCSIFeed()

Performing action = [1, 1]
/nmoved:  0.15080857169394182
Performing action = [1, 2]
/nmoved:  0.15127084909075825
Performing action = [1, 2]
/nmoved:  0.15468305824588285
Performing action = [2, 1]
/nmoved:  -0.15023496849343787
Performing action = [2, 2]
/nmoved:  -0.15078985639865658
Performing action = [1, 2]
/nmoved:  0.15092420130682682
Performing action = [1, 0]
/nmoved:  0.15004211973976775
Performing action = [1, 1]
/nmoved:  0.15126850661568528
Performing action = [2, 2]
/nmoved:  -0.15136616689447915
Performing action = [1, 1]
/nmoved:  0.15155650540571675
Performing action = [1, 1]
/nmoved:  0.1508659155980191
Performing action = [0, 1]
Performing action = [1, 2]
/nmoved:  0.15063630949392495
Performing action = [1, 2]
